In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import random
import string
import uuid
from datetime import date

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
gis_server = config['srv']['server']
pip_server = config['srv']['pip']
gis = config['db']['parksgis']
pip = config['db']['pip']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + pip_server +';Database=' + pip + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
pip_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
con_string = 'Driver={' + driver + '};Server=' + gis_server +';Database=' + gis + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
gis_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Set the prop id lookup field dictionary

In [6]:
#Define the dict of dicts that contains the feature classes and the translated column names
field_lookup = {'property_evw': {'propnum': 'gispropnum',
                                 'prop id': 'gispropnum', 
                                 'borough': 'department',
                                 'ampsdistrict': 'department',
                                 'prop name': 'signname',
                                 'site name': 'signname',
                                 'prop location': 'location',
                                 'site location': 'location',
                                 'acres': 'acres', 
                                 'jurisdiction': 'jurisdiction', 
                                 'typecategory': 'typecategory', 
                                 'featurestatus':'featurestatus', 
                                 'gisobjid':'gisobjid'}, 
                 'playground_evw': {'propnum': 'parentid', 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'signname', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},  
                 'zone_evw': {'propnum': 'parentid', 
                              'prop id': 'omppropid', 
                              'borough': 'department',
                              'ampsdistrict': 'department',
                              'site name': 'sitename', 
                              'site location': 'location', 
                              'acres': 'acres', 
                              'jurisdiction': 'jurisdiction', 
                              'featurestatus':'featurestatus', 
                              'gisobjid':'gisobjid'}, 

                 'unmapped_gisallsites_evw': {'propnum': 'gispropnum',
                                              'prop id': 'omppropid', 
                                              'borough': 'department',
                                              'ampsdistrict': 'department',
                                              'site name': 'name',
                                              'site location': 'location',
                                              'acres': 'acres', 
                                              'jurisdiction': 'jurisdiction'},
                 'schoolyard_to_playground_evw': {'propnum': 'gispropnum',
                                                  'prop id': 'gispropnum', 
                                                  'borough': 'department',
                                                  'ampsdistrict': 'department',
                                                  'prop name': 'signname',
                                                  'site name': 'signname',
                                                  'prop location': 'location',
                                                  'site location': 'location',
                                                  'acres': 'acres', 
                                                  'jurisdiction': 'jurisdiction', 
                                                  'featurestatus':'featurestatus'},
                 'greenstreet_evw': {'propnum': 'gispropnum', 
                                     'prop id': 'omppropid', 
                                     'borough': 'department',
                                     'ampsdistrict': 'department',
                                     'site name': 'sitename', 
                                     'site location': 'location', 
                                     'acres': 'acres', 
                                     'jurisdiction': 'jurisdiction', 
                                     'featurestatus':'featurestatus', 
                                     'gisobjid':'gisobjid'},
                 'golfcourse_evw': {'propnum': 'gispropnum', 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'name', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},
                 'restrictivedeclarationsite_evw': {'propnum': 'gispropnum',
                                                    'prop id': 'gispropnum', 
                                                    'borough': 'department',
                                                    'ampsdistrict': 'department',
                                                    'prop name': 'signname',
                                                    'site name': 'signname',
                                                    'prop location': 'location',
                                                    'site location': 'location',
                                                    'acres': 'acres', 
                                                    'jurisdiction': 'jurisdiction', 
                                                    'featurestatus':'featurestatus'}, 
                 'structure_evw': {'propnum': 'gispropnum',
                                   'prop id': 'omppropid', 
                                   'borough': 'department',
                                   'ampsdistrict': 'department',
                                   'prop name': 'description',
                                   'site name': 'description',
                                   'prop location': 'location',
                                   'site location': 'location',
                                   'jurisdiction': 'jurisdiction',  
                                   'featurestatus':'featurestatus', 
                                   'gisobjid':'gisobjid'}}

In [7]:
#Define a dict of source feature classes that map to the sourcefc value
sourcefc_lookup = {'property_evw': 'Property', 
                   'playground_evw': 'Playground', 
                   'zone_evw': 'Zone', 
                   'unmapped_gisallsites_evw': 'Unmapped',
                   'schoolyard_to_playground_evw': 'Schoolyard To Playground',
                   'greenstreet_evw': 'Greenstreet',
                   'golfcourse_evw': 'GolfCourse',
                   'restrictivedeclarationsite_evw': 'RestrictiveDeclarationSite',
                   'structure_evw': 'Structure'}

In [8]:
field_lookup_as = {f: ['['+ i[1] + '] as ' + '['+ i[0] + ']' for i in field_lookup[f].items()] for f in sourcefc_lookup }

## Original GIS Data

In [9]:
#Define the tables that will be queried and interacted with
gis_tables = ['property_evw', 'playground_evw', 'zone_evw', 'unmapped_gisallsites_evw', 
              'schoolyard_to_playground_evw', 'greenstreet_evw', 'golfcourse_evw', 
              'restrictivedeclarationsite_evw', 'structure_evw']

In [10]:
#Create the list of SQL Queries
gis_sql_list = ["select objectid, {}, '{}' as sourcefc from parksgis.dpr.{}"
                .format(' ,'.join(field_lookup_as[t]), sourcefc_lookup[t], t) 
                        for t in gis_tables]

In [11]:
#Create a dictionary with sources and dataframes that contain the original data
gis_df_list = {s: pd.read_sql(con = gis_engine, sql = q) for q, s in zip(gis_sql_list, gis_tables)}

## Execute all the stored procedures before starting

In [12]:
db_delete_sql = '''begin transaction

                       truncate table accessnewpip.dbo.tbl_ref_allsites_nosync
                       
                       truncate table accessnewpip.dbo.tbl_ref_allsites_audit
                       
                       truncate table accessnewpip.dbo.tbl_pip_allsites
                       
                       truncate table accessnewpip.dbo.tbl_pip_allsites_audit
                       
                       delete 
                       from accessnewpip.dbo.tbl_ref_allsites
                   commit'''

In [13]:
db_sql = ['exec accessnewpip.dbo.usp_i_tbl_temp_ref_allsites', 'exec accessnewpip.dbo.usp_m_tbl_ref_allsites',
          'exec accessnewpip.dbo.usp_m_tbl_ref_allsites_nosync', 'exec accessnewpip.dbo.usp_m_tbl_pip_allsites']

In [14]:
pip_con = pip_engine.connect()

In [15]:
#Loop through and execute the update queries
for q in [db_delete_sql] + db_sql:
    pip_con.execute(q)

In [16]:
pip_con.close()

## Original PIP Data

In [17]:
#Define the tables that will be queried and interacted with
pip_tables = ['tbl_ref_allsites', 'tbl_ref_allsites_audit', 'tbl_ref_allsites_nosync', 'tbl_pip_allsites', 
              'vw_pip_sync', 'allsites']

In [18]:
#Create the list of SQL Queries
pip_sql_list = ['select * from accessnewpip.dbo.'+ t for t in pip_tables]

In [19]:
#Create a list of dataframes with the original dat
pip_df_list = {s: pd.read_sql(con = pip_engine, sql = q) for q, s in zip(pip_sql_list, pip_tables)}

In [20]:
sql = 'select distinct [prop id] from accessnewpip.dbo.vw_pip_sync'

In [21]:
pip_sync_view = pd.read_sql(con = pip_engine, sql = sql)

In [22]:
existing_propid = list(pip_sync_view['prop id'])

## Invert the field lookup

In [23]:
inverted_field_lookup = {g:{v:k for k, v in field_lookup[g].items()} for g in gis_tables}

## Copy the DFs

In [24]:
n_rows = {g: random.randint(5, 10) for g in gis_tables}

In [25]:
gis_dfs_update_delete = {g: gis_df_list[g].copy().head(n_rows[g] * 2) for g in gis_tables}

In [26]:
gis_dfs_update = {g: gis_dfs_update_delete[g].copy().head(n_rows[g]) for g in gis_tables}

In [27]:
gis_dfs_delete = {g: gis_dfs_update_delete[g].copy().tail(n_rows[g]) for g in gis_tables}

In [28]:
gis_dfs_insert = {g: gis_df_list[g].copy().tail(n_rows[g]) for g in gis_tables}

## Perform DML Steps

### Delete Records

In [29]:
list_props = {g: list(gis_dfs_delete[g]['prop id']) for g in gis_tables}

In [30]:
where_list = {g: ','.join(f"'{p}'" for p in list(list_props[g])) for g in gis_tables}

In [31]:
#Define the common where clause using the prop id field lookup for each source GIS feature class
where = {f: str(r"where {} in({})").format(field_lookup[f]['prop id'], where_list[f]) for f in field_lookup}

In [32]:
#Loop through the GIS datasets, drop the _evw and create the queries that perform the deletes
sql = ['delete from parksgis.dpr.{} {}'.format(g.replace('_evw', ''), where[g]) for g in gis_tables]

In [33]:
gis_con = gis_engine.connect()

In [34]:
#Loop through and execute the delete queries
for q in sql:
    gis_con.execute(q)

In [35]:
#Create a list of dataframes with the data after deletes, notably all dfs should be empty
gis_df_post_delete = {g: gis_df_list[g][gis_df_list[g]['prop id'].isin(list_props[g])] for g in gis_tables}

### Update Records

In [36]:
update_col = 'site location'

In [37]:
update_val = 'Testing Updates'

In [38]:
list_props = {g: list(gis_dfs_update[g]['prop id']) for g in gis_tables}

In [39]:
where_list = {g: ','.join(f"'{p}'" for p in list(list_props[g])) for g in gis_tables}

In [40]:
#Define the common where clause using the prop id field lookup for each source GIS feature class
where = {f: str(r"where {} in({})").format(field_lookup[f]['prop id'], where_list[f]) for f in field_lookup}

In [41]:
set_value = {g: str(r"[{}] = '{}'").format(field_lookup[g][update_col], update_val) for g in gis_tables}

In [42]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['update parksgis.dpr.{} set {}  {}'.format(g.replace('_evw', ''), set_value[g], where[g]) for g in gis_tables]

In [43]:
#Loop through and execute the update queries
for q in sql:
    gis_con.execute(q)

In [44]:
#Create a list of dataframes with the data after deletes, notably all dfs should be empty
gis_df_post_update = {g: gis_df_list[g][gis_df_list[g]['prop id'].isin(list_props[g])] for g in gis_tables}

### Insert Non-Duplicate Records

In [45]:
#Create a function to generate prop ids
def gen_propid(sourcefc):
    boros = ['B','Q','M','R','X']

    boro = boros[random.randint(0,len(boros) - 1)]
    
    num = f'{random.randint(0, 999):03}'
    
    letters = list(string.ascii_uppercase)
    
    suffixes = ['', letters[random.randint(0,len(letters) - 1)]]
    
    suffix = suffixes[random.randint(0,len(suffixes) - 1)]
    
    suffix_num = f'{random.randint(0, 99):02}'
    
    if sourcefc == 'zone_evw':
        propid = boro + num + suffix + '-ZN' + suffix_num
        
    elif sourcefc == 'playground_evw':
        propid = boro + num + suffix + '-' + suffix_num
        
    elif sourcefc == 'greenstreet_evw':     
        propid = boro + 'Z' + num + suffix
        
    else:
        propid = boro + num + suffix
    
    while propid not in existing_propid:
        return propid
    
    else:
        gen_propid(sourcefc)

In [46]:
def get_value(sourcefc):
    for k, v in field_lookup[sourcefc].items():
        if k == 'prop id':
            return v

In [103]:
get_value('unmapped_gisallsites_evw')

'omppropid'

In [104]:
apply_cols['unmapped_gisallsites_evw']

['prop id']

In [47]:
apply_cols = {g:[k for k, v in field_lookup[g].items() if v in get_value(g)] for g in gis_tables }

In [107]:
gis_dfs_insert['unmapped_gisallsites_evw']

,objectid,gispropnum,omppropid,department,name,location,acres,jurisdiction,globalid
173,1017,Q162I07,X371R,Q-14,PARK STRIP,test,0.000,DPR,af4b69cb-b374-4b9c-b86b-743c68965506
174,1074,RZ17A,X710,R-02,GREENSTREET,None,0.000,DPR,8d474094-d6ae-4787-b5e6-b3c67155c11e
175,974,B457,M112D,B-05,DOSCHER ST. BLOCK ASSOCIATION,Doscher St.,0.085,DPR,d7335d4a-8bbd-478a-ae90-d6e3399884e8
176,1077,RZ65,None,R-03,GREENSTREET,DEWEY PLACE & GREAVES AVENUE,0.000,DPR,eba1a375-9f2c-4fac-bce8-1f14fb6c24ef
177,1139,RZ60,None,R-02,GREENSTREET,LILY POND & VERRAZANO-NARROWS BRIDGE,0.000,DPR,d92ecce2-a0b0-4246-8927-8853e2817bc0


In [108]:
for g in gis_tables:
    
    #Generate the prop ids for all relevant columns
    for c in apply_cols[g]:
        print(g, c)
        #gis_dfs_insert[g][c]

property_evw propnum
property_evw prop id
playground_evw prop id
zone_evw prop id
unmapped_gisallsites_evw prop id
schoolyard_to_playground_evw propnum
schoolyard_to_playground_evw prop id
greenstreet_evw prop id
golfcourse_evw prop id
restrictivedeclarationsite_evw propnum
restrictivedeclarationsite_evw prop id
structure_evw prop id


In [105]:
for g in gis_tables:
    
    #Take the existing max objectid and add that value to the current objectid
    objectid = gis_df_list[g].copy()['objectid'].max()
    gis_dfs_insert[g]['objectid'] = gis_dfs_insert[g].apply(lambda x: x['objectid'] + objectid, axis = 1)
    
    #Add a globalid column because it's required
    gis_dfs_insert[g]['globalid'] = gis_dfs_insert[g].apply(lambda x: str(uuid.uuid4()), axis = 1)
    
    #Generate the prop ids for all relevant columns
    for c in apply_cols[g]:
        gis_dfs_insert[g][c] = gis_dfs_insert[g].apply(lambda x: gen_propid(g), axis = 1)
     
    #Rename the columns to the original names
    gis_dfs_insert[g].rename(columns = field_lookup[g], inplace = True)
    
    #Remove duplicate columns and drop the sourcefc column
    gis_dfs_insert[g] = gis_dfs_insert[g].loc[:,~gis_dfs_insert[g].columns.duplicated()].drop(columns = ['sourcefc'])

KeyError: "['sourcefc'] not found in axis"

In [49]:
for g in gis_tables:
    fc = g.replace('_evw', '')
    gis_dfs_insert[g].to_sql(fc, gis_con, schema = 'dpr', if_exists = 'append', index = False)

In [101]:
apply_cols['unmapped_gisallsites_evw']

['prop id']

In [100]:
gis_dfs_insert['unmapped_gisallsites_evw']['omppropid']

173    X371R
174     X710
175    M112D
176     None
177     None
Name: omppropid, dtype: object

In [96]:
field_lookup['unmapped_gisallsites_evw']

{'propnum': 'gispropnum',
 'prop id': 'omppropid',
 'borough': 'department',
 'ampsdistrict': 'department',
 'site name': 'name',
 'site location': 'location',
 'acres': 'acres',
 'jurisdiction': 'jurisdiction'}

In [93]:
gis_dfs_insert['unmapped_gisallsites_evw']

,objectid,gispropnum,omppropid,department,name,location,acres,jurisdiction,globalid
173,1017,Q162I07,X371R,Q-14,PARK STRIP,test,0.000,DPR,af4b69cb-b374-4b9c-b86b-743c68965506
174,1074,RZ17A,X710,R-02,GREENSTREET,None,0.000,DPR,8d474094-d6ae-4787-b5e6-b3c67155c11e
175,974,B457,M112D,B-05,DOSCHER ST. BLOCK ASSOCIATION,Doscher St.,0.085,DPR,d7335d4a-8bbd-478a-ae90-d6e3399884e8
176,1077,RZ65,None,R-03,GREENSTREET,DEWEY PLACE & GREAVES AVENUE,0.000,DPR,eba1a375-9f2c-4fac-bce8-1f14fb6c24ef
177,1139,RZ60,None,R-02,GREENSTREET,LILY POND & VERRAZANO-NARROWS BRIDGE,0.000,DPR,d92ecce2-a0b0-4246-8927-8853e2817bc0


## Execute the PIP Sync

In [50]:
pip_con = pip_engine.connect()

In [51]:
#Loop through and execute the update queries
for q in db_sql:
    pip_engine.execute(q)

In [52]:
pip_con.close()

## Get the PIP and GIS Tables after

In [53]:
#Create a list of dataframes with the original dat
pip_df_list_after = {s: pd.read_sql(con = pip_engine, sql = q) for q, s in zip(pip_sql_list, pip_tables)}

In [54]:
#Create a dictionary with sources and dataframes that contain the original data
gis_df_list_after = {s: pd.read_sql(con = gis_engine, sql = q) for q, s in zip(gis_sql_list, gis_tables)}

# Conduct the Tests

In [55]:
test_results = []

### Merge the before and after dicts

In [56]:
gis_df_update_ck = {g: gis_dfs_update[g].merge(gis_df_list_after[g], how = 'left', on = 'prop id', indicator = True, suffixes = (None, '_y')) for g in gis_tables} 

In [57]:
gis_df_delete_ck = {g: gis_dfs_delete[g].merge(gis_df_list_after[g], how = 'left', on = 'prop id', indicator = True, suffixes = (None, '_y')) for g in gis_tables} 

In [58]:
gis_df_insert_ck = {g: gis_dfs_insert[g].merge(gis_df_list_after[g], how = 'left', left_on = field_lookup[g]['prop id'], right_on = 'prop id', indicator = True, suffixes = (None, '_y')) for g in gis_tables} 

## Validate that the row count of the deleted records matches

In [59]:
#The length of the records that are only in the deleted df (left_only) should be equal to the number of records in the delted df.
gis_delete_valid = {g: len(gis_df_delete_ck[g][gis_df_delete_ck[g]['_merge'] == 'left_only']) == len(gis_dfs_delete[g]) for g in gis_tables}

In [60]:
test_results.append({'Validation of Deletes in GIS evws' : all(v == True for v in gis_delete_valid.values())})

In [61]:
gis_insert_valid = {g: len(gis_df_insert_ck[g][gis_df_insert_ck[g]['_merge'] == 'both']) == len(gis_dfs_insert[g]) for g in gis_tables}

In [62]:
test_results.append({'Validation of Inserts into GIS evws' : all(v == True for v in gis_insert_valid.values())})

In [63]:
gis_update_valid = {g: len(gis_df_update_ck[g][gis_df_update_ck[g]['_merge'] == 'both']) == len(gis_dfs_insert[g]) for g in gis_tables}

In [64]:
test_results.append({'Validation of Updates into GIS evws' : all(v == True for v in gis_update_valid.values())})

In [65]:
gis_update_valid2 = {g: len(gis_df_update_ck[g][gis_df_update_ck[g][update_col] == update_val]) == len(gis_dfs_insert[g]) for g in gis_tables}

In [66]:
test_results.append({'Validation of Updates values into GIS evws' : all(v == True for v in gis_update_valid2.values())})

## Update the datetime column to date only

In [68]:
ck_date = date.today().strftime('%Y-%m-%d')

In [69]:
pip_df_list_after['tbl_ref_allsites']['created_date'] = pip_df_list_after['tbl_ref_allsites'].apply(lambda x: x['created_date'].strftime('%Y-%m-%d'), axis = 1)

In [70]:
pip_df_list_after['tbl_pip_allsites']['created_date'] = pip_df_list_after['tbl_pip_allsites'].apply(lambda x: x['created_date'].strftime('%Y-%m-%d'), axis = 1)

## Do the validation with Allsites

In [71]:
pip_df_update_ck = {g: gis_dfs_update[g].merge(pip_df_list_after['tbl_ref_allsites'][pip_df_list_after['tbl_ref_allsites']['sourcefc'] == sourcefc_lookup[g]], how = 'left', left_on = ['prop id'], right_on = ['Prop ID'], indicator = True, suffixes = (None, '_y')) for g in gis_tables} 

In [72]:
pip_df_update_audit_ck = {g: gis_dfs_update[g].merge(pip_df_list_after['tbl_ref_allsites_audit'][pip_df_list_after['tbl_ref_allsites_audit']['sourcefc'] == sourcefc_lookup[g]], how = 'left', left_on = ['prop id'], right_on = ['Prop ID'], indicator = True, suffixes = (None, '_y')) for g in gis_tables} 

In [73]:
pip_df_delete_ck = {g: gis_dfs_delete[g].merge(pip_df_list_after['tbl_ref_allsites'][pip_df_list_after['tbl_ref_allsites']['sourcefc'] == sourcefc_lookup[g]], how = 'left', left_on = ['prop id'], right_on = ['Prop ID'], indicator = True, suffixes = (None, '_y')) for g in gis_tables} 

In [74]:
pip_df_insert_ck = {g: gis_dfs_insert[g].merge(pip_df_list_after['tbl_ref_allsites'][pip_df_list_after['tbl_ref_allsites']['sourcefc'] == sourcefc_lookup[g]], how = 'left', left_on = [field_lookup[g]['prop id']], right_on = ['Prop ID'], indicator = True, suffixes = (None, '_y')) for g in gis_tables}

In [75]:
pip_df_insert_ck2 = {g: gis_dfs_insert[g].merge(pip_df_list_after['tbl_pip_allsites'], how = 'left', left_on = [field_lookup[g]['prop id']], right_on = ['prop id'], indicator = True, suffixes = (None, '_y')) for g in gis_tables}

## Check the validity for all the PIP Tests

In [76]:
pip_df_update_valid = {g: len(pip_df_update_ck[g][(pip_df_update_ck[g]['_merge'] == 'both') & (pip_df_update_ck[g][update_col] == update_val)]) == len(gis_dfs_update[g]) for g in gis_tables}

In [77]:
test_results.append({'Validation of Updates with values in tbl_ref_allsites' : all(v == True for v in gis_delete_valid.values())})

In [78]:
pip_df_update_audit_valid = {g: len(pip_df_update_audit_ck[g][(pip_df_update_audit_ck[g]['_merge'] == 'both') & (pip_df_update_audit_ck[g]['created_date'] == ck_date)]) == len(gis_dfs_update[g]) for g in gis_tables}

In [79]:
test_results.append({'Validation of Updates in tbl_ref_allsites_audit' : all(v == True for v in gis_delete_valid.values())})

In [80]:
pip_df_delete_valid = {g: len(pip_df_delete_ck[g][(pip_df_delete_ck[g]['_merge'] == 'both') & (pip_df_delete_ck[g]['gis_deleted'] == 1)]) == len(gis_dfs_delete[g]) for g in gis_tables}

In [81]:
test_results.append({'Validation of Deletes tbl_ref_allsites with flag' : all(v == True for v in gis_delete_valid.values())})

In [82]:
pip_df_insert_valid = {g: len(pip_df_insert_ck[g][(pip_df_insert_ck[g]['_merge'] == 'both') & (pip_df_insert_ck[g]['created_date'] == ck_date)]) == len(gis_dfs_insert[g]) for g in gis_tables}

In [83]:
test_results.append({'Validation of Inserts into tbl_ref_allsites' : all(v == True for v in gis_delete_valid.values())})

In [84]:
pip_df_insert_valid2 = {g: len(pip_df_insert_ck2[g][(pip_df_insert_ck2[g]['_merge'] == 'both') & (pip_df_insert_ck[g]['created_date'] == ck_date)]) == len(gis_dfs_insert[g]) for g in gis_tables if g not in 'structure_evw'}

In [85]:
test_results.append({'Validation of Inserts into tbl_pip_allsites, except structures' : all(v == True for v in gis_delete_valid.values())})

In [86]:
test_results

[{'Validation of Deletes in GIS evws': True},
 {'Validation of Inserts into GIS evws': False},
 {'Validation of Updates into GIS evws': False},
 {'Validation of Updates values into GIS evws': False},
 {'Validation of Updates with values in tbl_ref_allsites': True},
 {'Validation of Updates in tbl_ref_allsites_audit': True},
 {'Validation of Deletes tbl_ref_allsites with flag': True},
 {'Validation of Inserts into tbl_ref_allsites': True},
 {'Validation of Inserts into tbl_pip_allsites, except structures': True}]